The data used here can be found [here](https://www.pxweb.bfs.admin.ch/Selection.aspx?px_language=fr&px_db=px-x-0702000000_104&px_tableid=px-x-0702000000_104\px-x-0702000000_104.px&px_type=PX).

We gathered the data from 2013-2015 (2016) is not available. However, the names of the municipalities are those from the year 2016 (or later). In ordrer to have some data for the right municipality names, we assume that the ratio cow/inhabitant doesn't change between the municipalities that merge together.

In [117]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import re

In [118]:
df = pd.read_csv('data_to_be_processed/cows.csv', encoding='latin_1', skiprows=2, delimiter=';')

In [119]:
df = df.drop(['Système d\'exploitation'], axis=1)
df.columns = ['year', 'commune', 'cows']
df.head()

,year,commune,cows
0,2013,- Suisse,1557474
1,2013,- Zürich,93233
2,2013,>> Bezirk Affoltern,9315
3,2013,......0001 Aeugst am Albis,622
4,2013,......0002 Affoltern am Albis,596


### Preprocessing

In [120]:
#remove districts
df = df.loc[(df.commune.str.contains('>>') == False)]
#remove cantons
df = df.loc[(df.commune.str.contains('- ') == False)]
#remove swiss total
df = df[df['commune'] != '- Suisse']
#rewrite commune name
df['id'], df['commune'] = df['commune'].str.split(' ', 1).str
regex = re.compile(r"\.{6}")
df['id'].replace(to_replace=regex, value='', inplace=True, regex=True)
df['id'] = df['id'].astype(int)
df.head()

,year,commune,cows,id
3,2013,Aeugst am Albis,622,1
4,2013,Affoltern am Albis,596,2
5,2013,Bonstetten,314,3
6,2013,Hausen am Albis,1368,4
7,2013,Hedingen,312,5


In [123]:
df[df.year == 2013].shape

(2330, 4)

In [83]:
f = pd.read_csv("data_to_be_processed/fusion_mappings.csv")
f = f[f.date.str.contains('2016')]
f.head()
df[df['id'] == 310]
f.shape

(54, 5)

In [84]:
f[f['post_name'] == 'Rapperswil (BE)']

,pre_id,pre_name,post_id,post_name,date
700,532,Bangerten,310,Rapperswil (BE),01.01.2016
783,310,Rapperswil (BE),310,Rapperswil (BE),01.01.2016


In [85]:
fu = pd.merge(f, df, left_on='post_name', right_on='commune', how='right')
fu['commune'] = fu['pre_name'].fillna(fu['commune'])

fu[fu['id'] == 310]

,pre_id,pre_name,post_id,post_name,date,year,commune,cows,id
6,532.0,Bangerten,310.0,Rapperswil (BE),01.01.2016,2013,Bangerten,1716,310
7,310.0,Rapperswil (BE),310.0,Rapperswil (BE),01.01.2016,2013,Rapperswil (BE),1716,310
8,532.0,Bangerten,310.0,Rapperswil (BE),01.01.2016,2014,Bangerten,1679,310
9,310.0,Rapperswil (BE),310.0,Rapperswil (BE),01.01.2016,2014,Rapperswil (BE),1679,310
10,532.0,Bangerten,310.0,Rapperswil (BE),01.01.2016,2015,Bangerten,1695,310
11,310.0,Rapperswil (BE),310.0,Rapperswil (BE),01.01.2016,2015,Rapperswil (BE),1695,310


In [87]:
fu = pd.merge(f, df, left_on='post_name', right_on='commune', how='right')
fu['commune'] = fu['pre_name'].fillna(fu['commune'])
fu['compute_ratio_id'] = fu['id']
fu['id'] = fu['pre_id'].fillna(fu['id'])
fu['id'] = fu['id'].astype(int)
# fu['compute_ratio_id'] = fu['compute_ratio_id'].astype(int)
fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1, inplace=True)
fu.head()

,year,commune,cows,id,compute_ratio_id
0,2013,Kyburg,1573,175,296
1,2013,Illnau-Effretikon,1573,174,296
2,2014,Kyburg,1590,175,296
3,2014,Illnau-Effretikon,1590,174,296
4,2015,Kyburg,1675,175,296


In [15]:
# tmp = df[df['year'] == 2015]
# # tmp = tmp.drop('year', axis=1)

# fu = pd.merge(f, df, left_on='pre_name', right_on='commune', how='outer')
# fu['commune'] = fu['post_name'].fillna(fu['commune'])
# fu['id'] = fu['post_id'].fillna(fu['id'])
# fu['id'] = fu['id'].astype(int)
# fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1, inplace=True)
# fu = fu.groupby(['id', 'commune', 'sex', 'nationality', 'age'], as_index=False).sum()

In [16]:
# def create_2016_data(data):
# regex = re.compile(r"\d\d\.\d\d\.")
# f['date'].replace(to_replace=regex, value='', inplace=True, regex=True)
# fu = pd.merge(f, tmp, left_on='post_name', right_on='commune', how='right')
# # fu['commune'] = fu['post_name'].fillna(fu['commune'])
# fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1, inplace=True)
# fu = fu.groupby(['commune', 'id', 'nationality', 'age'], as_index=False).sum()
#     return fu

In [17]:
# fu[fu.date.astype(float) < fu.year] # commune = pre_name, id = pr_id

# #final: compute_ratio_id, id, commune, cows, year
# #compute_ratio_id = id of municipality of current name (column 'id')
# #id, commune : id of municipality adjusted by the years (pre_id if mun has merged ( date > year), else id)
# #year = column year
# fu['compute_ratio_id'] = fu['id']
# #check if equality is in the right direction
# fu['id'] = fu.apply((lambda x: x.pre_id if (float(x.date) > x.year) else x.id ), axis=1 )
# fu['commune'] = fu.apply((lambda x: x.pre_name if (float(x.date) > x.year) else x.commune ), axis=1 )

In [108]:
fu.shape

(7095, 5)

In [19]:
# fu = fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1)

In [88]:
# fu['id'] = fu['id'].astype(int)
for year in range (2013, 2017):    
    #filter out by year
    if(year == 2016):
        tmp = fu[fu['year'] == 2015]
        tmp = tmp.drop('year', axis=1)
    else:
        tmp = fu[fu['year'] == year]
        tmp = tmp.drop('year', axis=1)
    tmp.to_csv('municipalities/%d/data_cows.csv' % year, index=False)
tmp.head()

,commune,cows,id,compute_ratio_id
4,Kyburg,1675,175,296
5,Illnau-Effretikon,1675,174,296
10,Bangerten,1695,532,310
11,Rapperswil (BE),1695,310,310
18,Niederösch,1016,417,405


In [100]:
tmp.shape
tmp[tmp.isnull().any(1)]

,commune,cows,id,compute_ratio_id


In [93]:
mun = pd.read_csv('municipalities/2016/data_commune.csv')

In [94]:
mun.shape

(2324, 9)

In [104]:
foo = pd.merge(tmp, mun, left_on='id', right_on='id', how='outer')


(2365, 12)

In [124]:
len(foo.id.unique())

2324

In [127]:
foo[foo.id.duplicated()]

,commune_x,cows,id,compute_ratio_id,commune_y,total_inhabitants,percentage_18,percentage_40,percentage_65,percentage_100,percentage_men,percentage_swiss
1,175,0,175,175,Kyburg,407,0.21,0.20,0.45,0.13,0.51,0.90
3,174,0,174,174,Illnau-Effretikon,16327,0.18,0.30,0.33,0.20,0.49,0.74
5,532,0,532,532,Bangerten,161,0.17,0.23,0.45,0.16,0.45,0.93
8,417,0,417,417,Niederösch,235,0.25,0.21,0.37,0.18,0.48,0.99
10,419,0,419,419,Oberösch,104,0.18,0.21,0.38,0.22,0.54,0.95
13,978,0,978,978,Hermiswil,96,0.20,0.26,0.34,0.20,0.48,0.94
16,2014,0,2014,2014,Dompierre (FR),1022,0.24,0.34,0.31,0.10,0.52,0.71
18,2013,0,2013,2013,Domdidier,3064,0.23,0.30,0.34,0.14,0.51,0.76
20,2040,0,2040,2040,Russy,216,0.18,0.29,0.37,0.17,0.53,0.97
22,2024,0,2024,2024,Léchelles,804,0.21,0.39,0.30,0.11,0.51,0.81
